# Week 3 Day 2 pt. 2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/defreez/cs356-notebooks/blob/main/notebooks/landmarks.ipynb)

In [ ]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
fastbook.setup_book()
from fastai.vision import *

from fastai.vision.widgets import *

In [2]:
from fastbook import *
import xml.etree.ElementTree as ET 

In [ ]:
path = untar_data('http://dlib.net/files/data/ibug_300W_large_face_landmark_dataset.tar.gz')

In [ ]:
path.ls()

In [6]:
def img2pts(x): 
    return Path(f'{str(x)[:-3]}pts')

In [7]:
def get_landmarks(f):
    return tensor(np.genfromtxt(img2pts(f), skip_header=3, skip_footer=1))

In [8]:
def get_image_files_skip(p):
    return [x for x in get_image_files(p) if "mirror" not in str(x)]

In [9]:
data = DataBlock(
    blocks=(ImageBlock, PointBlock),
    get_items=get_image_files_skip,
    get_y=get_landmarks,
    item_tfms=Resize(480),
)

In [10]:
dls = data.dataloaders(path/'helen')

In [ ]:
dls.show_batch(max_n=9)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(-1, 1))

In [ ]:
learn.lr_find()

In [ ]:
lr = 0.004
learn.fine_tune(, lr)

In [153]:
learn = load_learner(gdrive / '480-helen-32.pkl')

In [ ]:
learn.show_results(ds_idx=1, max_n=3)

In [ ]:
uploader = widgets.FileUpload()
uploader

In [190]:
img = PILImage.create(uploader.data[0])

In [ ]:
y = learn.predict(img)

In [193]:
img = img.resize((480, 480))

In [194]:
pnt_img = TensorImage(img)

In [ ]:
ctx = pnt_img.show()
y[0].show(ctx=ctx)

In [ ]:
learn.export(gdrive/'landmarks.pkl')

In [16]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  img = Image.open("photo.jpg")
  img = img.crop((80, 0, 560, 480))
  img.save("photo.jpg")

  uploader = SimpleNamespace(data = ['photo.jpg'])
  img = PILImage.create(uploader.data[0])
  y = learn.predict(img)
  pnt_img = TensorImage(img)
  ctx = pnt_img.show()
  y[0].show(ctx=ctx)
  
  # Show the image which was just taken.
  #display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))